# Stephen's Code for Appproach 2 Events Classification

# Labraries Imports

In [1]:
import csv
import re
import numpy as np

In [2]:
import nltk
from nltk.corpus import stopwords
from nltk.stem import PorterStemmer               #WordNetLemmatizer
nltk.download('stopwords')

[nltk_data] Downloading package stopwords to /home/steve/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


True

In [3]:
from sklearn.datasets import load_files

from sklearn.ensemble import RandomForestClassifier
#from sklearn.linear_model import LogisticRegression
#from sklearn.linear_model import LinearRegression
from sklearn.neighbors import KNeighborsClassifier
from sklearn.svm import LinearSVC
from sklearn.naive_bayes import BernoulliNB
from sklearn.neighbors import NearestCentroid
from sklearn.metrics import classification_report, confusion_matrix,accuracy_score
from sklearn.model_selection import train_test_split

# DataSet Import

In [4]:
train_text = []
train_categories = []

with open("final_data.csv", "r") as file:
    csv_reader = csv.reader(file, delimiter=',')
    line_count = 0
    for rows in csv_reader:
        if line_count == 0:
            pass
        else:
            train_text.append(rows[1])
            train_categories.append(rows[3])
        line_count += 1    

# Text Preproccessing

In [5]:
train_categories = [x.lower() for x in train_categories]

def normalize(train_text):
    
    stemmer = PorterStemmer()
    
    new_traintext = []
    
    for t_text in train_text:
        
        text = t_text
        
        #Remove reply identifiers. Ex: "Reply to @Someone: ...."
        text = re.sub(r"Reply to @\w.*:", "", text)
        
        #Remove IDs. Ex: "@someone"
        text = re.sub(r"@\w+\s*", "", text)
        
        #Remove all special characters 
        text = re.sub(r'\W', ' ', text)
        
        #Remove all single characters
        text = re.sub(r'\s+[a-zA-Z]\s+', ' ', text)
        
        # Remove single characters from the start
        text = re.sub(r'\^[a-zA-Z]\s+', ' ', text) 
    
        #Substituting multiple spaces with single space
        text = re.sub(r'\s+', ' ', text, flags=re.I)
        
        text = re.sub(r'^b\s+', '', text)
        
        text = text.lower()
        
        text = text.split()

        text = [stemmer.stem(word) for word in text]
        text = ' '.join(text)
    
        new_traintext.append(text)
        
    return new_traintext



train_text = normalize(train_text)

# A Text Numerical value conversion

In [6]:
from sklearn.feature_extraction.text import TfidfVectorizer


tfidfconverter = TfidfVectorizer(
    max_features = 1500,
    min_df = 5,
    max_df = 0.7,
    
    stop_words = stopwords.words('english')
)

X = tfidfconverter.fit_transform(train_text).toarray()


# Save the vocabulary to calculate TF/IDFs on the fly from new tweets
vocab = tfidfconverter.vocabulary_


# Data Splitting: Train Set And Test Set

In [7]:
#from sklearn.model_selection import train_test_split

X_train, X_test, y_train, y_test = train_test_split(X, train_categories ,test_size=0.2, random_state=0)

# Models Training

In [8]:
classifier0 = RandomForestClassifier(n_estimators=1000, random_state=0)
classifier0.fit(X_train, y_train) 


RandomForestClassifier(bootstrap=True, class_weight=None, criterion='gini',
            max_depth=None, max_features='auto', max_leaf_nodes=None,
            min_impurity_decrease=0.0, min_impurity_split=None,
            min_samples_leaf=1, min_samples_split=2,
            min_weight_fraction_leaf=0.0, n_estimators=1000, n_jobs=None,
            oob_score=False, random_state=0, verbose=0, warm_start=False)

In [9]:
classifier1 = KNeighborsClassifier(n_neighbors=5)
classifier1.fit(X_train, y_train) 


KNeighborsClassifier(algorithm='auto', leaf_size=30, metric='minkowski',
           metric_params=None, n_jobs=None, n_neighbors=5, p=2,
           weights='uniform')

In [10]:
classifier2 =  LinearSVC()
classifier2.fit(X_train, y_train) 

LinearSVC(C=1.0, class_weight=None, dual=True, fit_intercept=True,
     intercept_scaling=1, loss='squared_hinge', max_iter=1000,
     multi_class='ovr', penalty='l2', random_state=None, tol=0.0001,
     verbose=0)

In [11]:
classifier3 = BernoulliNB()
classifier3.fit(X_train, y_train) 

BernoulliNB(alpha=1.0, binarize=0.0, class_prior=None, fit_prior=True)

In [19]:
classifier4 = NearestCentroid()
classifier4.fit(X_train, y_train) 


NearestCentroid(metric='euclidean', shrink_threshold=None)

# Models Evaluation

In [12]:
y_predict = classifier0.predict(X_test)

print(accuracy_score(y_test,y_predict))
print(confusion_matrix(y_test,y_predict))
print(classification_report(y_test,y_predict))


0.9128386336866903
[[  0   0   0 ...   0   0   0]
 [  0   0   2 ...   0   0   0]
 [  0   0 330 ...   0   1   0]
 ...
 [  0   0   0 ...  21   0   0]
 [  0   0   1 ...   1 112   0]
 [  0   0   0 ...   0   0   2]]
                                   precision    recall  f1-score   support

                        accidents       0.00      0.00      0.00         1
          acts of violence or war       0.00      0.00      0.00         5
      armed conflicts and attacks       0.92      0.96      0.94       345
                 arts and culture       0.83      0.78      0.80        67
           business and economics       0.97      0.81      0.88        42
             business and economy       0.88      0.90      0.89        51
celebrity and human interest news       0.35      0.45      0.39        20
                        disasters       0.90      0.89      0.90        72
          disasters and accidents       0.88      0.88      0.88        33
                        elections     

/home/steve/anaconda3/lib/python3.7/site-packages/sklearn/metrics/classification.py:1143: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples.
  'precision', 'predicted', average, warn_for)


In [13]:
y_predict = classifier1.predict(X_test)

print(accuracy_score(y_test,y_predict))
print(confusion_matrix(y_test,y_predict))
print(classification_report(y_test,y_predict))

0.8130153121319199
[[  0   0   0 ...   0   0   0]
 [  0   0   3 ...   0   1   0]
 [  0   0 333 ...   0   0   0]
 ...
 [  0   0   0 ...  21   0   0]
 [  0   0   4 ...   2  95   0]
 [  0   0   0 ...   0   0   2]]
                                   precision    recall  f1-score   support

                        accidents       0.00      0.00      0.00         1
          acts of violence or war       0.00      0.00      0.00         5
      armed conflicts and attacks       0.84      0.97      0.90       345
                 arts and culture       0.27      0.82      0.41        67
           business and economics       0.92      0.79      0.85        42
             business and economy       0.90      0.84      0.87        51
celebrity and human interest news       0.37      0.50      0.43        20
                        disasters       0.92      0.90      0.91        72
          disasters and accidents       0.91      0.88      0.89        33
                        elections     

/home/steve/anaconda3/lib/python3.7/site-packages/sklearn/metrics/classification.py:1143: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples.
  'precision', 'predicted', average, warn_for)


In [14]:
y_predict = classifier2.predict(X_test)

print(accuracy_score(y_test,y_predict))
print(confusion_matrix(y_test,y_predict))
print(classification_report(y_test,y_predict))

0.9178445229681979
[[  0   0   0 ...   0   0   0]
 [  0   0   3 ...   0   0   0]
 [  0   0 329 ...   0   0   0]
 ...
 [  0   0   0 ...  21   0   0]
 [  0   0   0 ...   1 108   0]
 [  0   0   0 ...   0   0   2]]
                                   precision    recall  f1-score   support

                        accidents       0.00      0.00      0.00         1
          acts of violence or war       0.00      0.00      0.00         5
      armed conflicts and attacks       0.96      0.95      0.96       345
                 arts and culture       0.84      0.79      0.82        67
           business and economics       0.95      0.83      0.89        42
             business and economy       0.94      0.88      0.91        51
celebrity and human interest news       0.60      0.45      0.51        20
                        disasters       0.89      0.92      0.90        72
          disasters and accidents       0.90      0.85      0.88        33
                        elections     

/home/steve/anaconda3/lib/python3.7/site-packages/sklearn/metrics/classification.py:1143: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples.
  'precision', 'predicted', average, warn_for)


In [15]:
y_predict = classifier3.predict(X_test)

print(accuracy_score(y_test,y_predict))
print(confusion_matrix(y_test,y_predict))
print(classification_report(y_test,y_predict))

0.8630742049469965
[[  0   0   0 ...   0   0   0]
 [  0   0   4 ...   0   0   0]
 [  0   0 337 ...   0   0   0]
 ...
 [  0   0   1 ...   0   4   0]
 [  0   0   3 ...   0 117   0]
 [  0   0   0 ...   0   1   0]]
                                   precision    recall  f1-score   support

                        accidents       0.00      0.00      0.00         1
          acts of violence or war       0.00      0.00      0.00         5
      armed conflicts and attacks       0.82      0.98      0.89       345
                 arts and culture       0.64      0.61      0.63        67
           business and economics       0.96      0.62      0.75        42
             business and economy       0.95      0.76      0.85        51
celebrity and human interest news       0.00      0.00      0.00        20
                        disasters       0.78      0.86      0.82        72
          disasters and accidents       0.84      0.64      0.72        33
                        elections     

/home/steve/anaconda3/lib/python3.7/site-packages/sklearn/metrics/classification.py:1143: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples.
  'precision', 'predicted', average, warn_for)


In [20]:
y_predict = classifier4.predict(X_test)

print(accuracy_score(y_test,y_predict))
print(confusion_matrix(y_test,y_predict))
print(classification_report(y_test,y_predict))

0.766489988221437
[[  1   0   0 ...   0   0   0]
 [  0   2   2 ...   0   0   0]
 [  0   1 274 ...   0   0   0]
 ...
 [  0   0   0 ...  20   0   0]
 [  0   2   3 ...   3  96   0]
 [  0   0   0 ...   0   0   2]]
                                   precision    recall  f1-score   support

                        accidents       0.09      1.00      0.17         1
          acts of violence or war       0.13      0.40      0.20         5
      armed conflicts and attacks       0.82      0.79      0.81       345
                 arts and culture       0.69      0.54      0.61        67
           business and economics       0.86      0.76      0.81        42
             business and economy       0.84      0.73      0.78        51
celebrity and human interest news       0.25      0.50      0.33        20
                        disasters       0.97      0.47      0.64        72
          disasters and accidents       0.77      0.82      0.79        33
                        elections      

/home/steve/anaconda3/lib/python3.7/site-packages/sklearn/metrics/classification.py:1143: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples.
  'precision', 'predicted', average, warn_for)


# Automatic Testing of new Tweets

In [22]:
# Input
tweet1 = "there will be a HUGE football game tonight!"
tweet2 = "wow .. awesome stuff we saw tonight!"
tweet3 = "A new actress will feature in a new musical"

input_list = [tweet1, tweet2, tweet3]

# Normalize
normalized_input = normalize(input_list)

# Extract features
vectorizer = TfidfVectorizer(
    max_features=1500,
    min_df=5,
    max_df=0.7,
    stop_words=stopwords.words('english'),
    vocabulary=vocab # use the vocabulary from the training set
)

features = vectorizer.fit_transform(normalized_input)

# Prediction
predictions = classifier0.predict(features)

# Output
for i in range(len(input_list)):
    print(input_list[i], "=>", predictions[i])

there will be a HUGE football game tonight! => sports news
wow .. awesome stuff we saw tonight! => non_event
A new actress will feature in a new musical => business and economy


# User Interact with Models 

In [ ]:
# Extract features
vectorizer = TfidfVectorizer(
    max_features=1500,
    min_df=5,
    max_df=0.7,
    stop_words=stopwords.words('english'),
    vocabulary=vocab # use the vocabulary from the training set
)

while(True):
    text = input("Enter a Text of Your Choice: ")
    normalized = normalize([text])
    features = vectorizer.fit_transform(normalized)

# Prediction
    prediction = classifier0.predict(features)
    #prediction = classifier1.predict(features)
    #prediction = classifier2.predict(features)
    #prediction = classifier3.predict(features)
    #prediction = classifier4.predict(features)

    print("Category: " , prediction[0])
  

Enter a Text of Your Choice: Dr. Jelena Mitrovic, please can you evaluate my code today
Category:  non_event
Enter a Text of Your Choice: There is a University meeting today
Category:  non_event
